In [1]:
suppressPackageStartupMessages(suppressWarnings({
    library("ggplot2")
    library(dplyr)
    library(tidyr)
    library(ComplexHeatmap)
    library(tibble)
    library(RColorBrewer) 
    library(scales)
    library(circlize)
}))


In [2]:
profile_file_path <- file.path("../../data/CP_feature_select/profiles/features_selected_profile.parquet")
figure_path <- file.path("../figures/")
if (!dir.exists(figure_path)) {
    dir.create(figure_path, recursive = TRUE)
}

profile <- arrow::read_parquet(profile_file_path, col_select = c("Metadata_dose", "Metadata_Well", "Metadata_FOV"))

# arrange by well_fov
profile <- profile %>% arrange(Metadata_Well) %>% 
    select(Metadata_Well, Metadata_dose)
# remove duplicates
profile <- profile %>% 
    group_by(Metadata_Well) %>% 
    summarise(Metadata_dose = unique(Metadata_dose)) %>% 
    ungroup()

In [3]:
profile_file_path <- file.path("../../data/CP_aggregated/profiles/aggregated_profile.parquet")
df <- arrow::read_parquet(profile_file_path) %>% arrange(Metadata_Well)
df <- df %>% 
    left_join(profile, by = c("Metadata_Well" = "Metadata_Well"), relationship ="many-to-many")
# transform the data to standard scalar (-1, 1) format 
for (i in 1:ncol(df)) {
    # make sure the column is not metadata
    if (grepl("Metadata_", colnames(df)[i])) {
        next
    }
    if (is.numeric(df[[i]])) {
        df[[i]] <- rescale(df[[i]], to = c(-1, 1))
    }
}
# map each of the Time points to the actual timepoint
df$Metadata_Time <- as.numeric(df$Metadata_Time * 60 /2 )
head(df)

Metadata_Well,Metadata_Time,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Solidity,⋯,Nuclei_Texture_DifferenceVariance_CL_488_2_3_01_256,Nuclei_Texture_DifferenceVariance_CL_561_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CL_488_1_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CL_561_3_01_256,Nuclei_Texture_InverseDifferenceMoment_DNA_3_00_256,Nuclei_Texture_SumAverage_CL_488_1_3_03_256,Nuclei_Texture_SumAverage_CL_488_2_3_03_256,Nuclei_Texture_SumAverage_CL_561_3_02_256,Nuclei_Texture_SumAverage_DNA_3_00_256,Metadata_dose
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
C-02,0,0.1599026,-0.2683816,0.6725849,-0.01858689,0.010434170,0.4340497,0.0201994556,0.6020634,⋯,0.8918608,0.48462306,0.7119413,0.9339702,1,-0.8715191,0.6453382,0.6725822,-1,0
C-02,30,0.6041667,-0.4071476,0.6322834,0.12042897,0.165945362,0.7379347,0.3479018612,0.6692069,⋯,0.7339102,0.05786692,0.8556818,0.7988166,1,-0.9281102,0.6453382,0.8108737,-1,0
C-02,60,0.6866883,-0.3663201,0.7198465,0.01725914,0.118910230,0.7619172,0.1290836186,0.6573529,⋯,0.0856251,0.03735143,0.8807505,0.7924323,1,-0.9668016,0.6453382,0.8077566,-1,0
C-02,90,0.7107684,-0.2566143,0.8906667,-0.08704013,-0.002212294,0.7193351,0.0007231936,0.6183350,⋯,-0.4517980,-0.05870297,0.8721389,0.7604674,1,-0.9642729,0.6453382,0.8246567,-1,0
C-02,120,0.7123918,-0.2635095,0.9555743,-0.09591548,0.005183087,0.7937192,-0.0280002259,0.5861652,⋯,-1.0000000,-0.09307631,0.7735032,0.7628205,1,-0.9553412,-1.0000000,0.8535058,-1,0
C-02,150,0.7941017,-0.2258602,0.9378451,-0.20795765,-0.034856004,0.8249529,-0.1956768252,0.5574731,⋯,-1.0000000,-0.13931367,0.7635222,0.7171510,1,-0.9828635,-1.0000000,0.8845678,-1,0


In [4]:
# complex heatmap does not compare across heatmaps the scale so we must set it manually
# for more information see:
# https://github.com/jokergoo/EnrichedHeatmap/issues/7

# we will set the color scale the same way that ComplexHeatmap does
# The automatically generated colors map from the minus and plus 99^th of
# the absolute values in the matrix.


global_across_dose_99th_min <- df %>% 
    select(-Metadata_Well, -Metadata_dose, -Metadata_Time) %>%
    summarise(across(everything(), ~ quantile((.), 0.01, na.rm = TRUE))) %>%
    unlist() %>%
    min(na.rm = TRUE)
global_across_dose_99th_max <- df %>%
    select(-Metadata_Well, -Metadata_dose, -Metadata_Time) %>%
    summarise(across(everything(), ~ quantile((.), 0.99, na.rm = TRUE))) %>%
    unlist() %>%
    max(na.rm = TRUE)

print(global_across_dose_99th_min)
print(global_across_dose_99th_max)
col_fun = circlize::colorRamp2(c(global_across_dose_99th_min, 0, global_across_dose_99th_max), c("blue","white", "red"))

[1] -1
[1] 1


In [5]:
# get the list of features
features <- colnames(df)
features <- features[!features %in% c("Metadata_Well", "Metadata_dose", "Metadata_Time")]
features <- as.data.frame(features)
# split the features by _ into multiple columns
features <- features %>% 
    separate(features, into = c("Compartment", "Measurement", "Metric", "Extra", "Extra1", "Extra2", "Extra3"), sep = "_", extra = "merge", fill = "right")
# clean up the features columns
# if Extra is NA then replace with None
features$Extra[is.na(features$Extra)] <- "None"
# if extra is a number then replace with None
features$Extra[grepl("^[0-9]+$", features$Extra)] <- "None"
# replace all other NAs with None
features$Extra1[is.na(features$Extra1)] <- "None"
features$Extra2[is.na(features$Extra2)] <- "None"
# change extra to None if X or Y
features$Extra[features$Extra == "X"] <- "None"
features$Extra[features$Extra == "Y"] <- "None"
# drop the Adjacent channel
features$Extra[features$Extra == "Adjacent"] <- "None"
# if extra1 is 488 then add extra2 to Extra1
features$Extra1[features$Extra1 == "488"] <- paste0(features$Extra1[features$Extra1 == "488"], "_", features$Extra2[features$Extra1 == "488"])
# if extra1 id CL then add extra1 to Extra
features$Extra[features$Extra == "CL"] <- paste0(features$Extra[features$Extra == "CL"], "_", features$Extra1[features$Extra == "CL"])

features <- features %>% 
    rename(Channel = Extra) %>%
    select(-Extra1, -Extra2)
# rename channel names to replace "_" with " "
features$Channel <- gsub("CL_488_1", "CL 488_1", features$Channel)
features$Channel <- gsub("CL_488_2", "CL 488_2", features$Channel)
features$Channel <- gsub("CL_561", "CL 561", features$Channel)

# time color function
time_col_fun = colorRamp2(
    c(min(unique(df$Metadata_Time)), max(unique(df$Metadata_Time))), c("white", "purple")
    )

column_anno <- HeatmapAnnotation(
    Time = unique(df$Metadata_Time),
    show_legend = TRUE,
    annotation_name_gp = gpar(fontsize = 2),
    annotation_legend_param = list(
        title_position = "topcenter", 
        title_gp = gpar(fontsize = 16, angle = 0, fontface = "bold", hjust = 1.0),
        labels_gp = gpar(fontsize = 16, 
        title = gpar(fontsize = 16))),
    col = list(
        Time = time_col_fun
    )
)

# compartment row annotation
row_compartment = rowAnnotation(
    Object = features$Compartment,
        show_legend = TRUE,
    # change the legend titles
    annotation_legend_param = list(
        title_position = "topcenter", 
        title_gp = gpar(fontsize = 16, angle = 0, fontface = "bold", hjust = 1.0),
        labels_gp = gpar(fontsize = 16, 
        title = gpar(fontsize = 16))),
    annotation_name_side = "bottom",
    annotation_name_gp = gpar(fontsize = 16),
    # color
    col = list(
        Object = c(
            "Cells" = "#B000B0", 
            "Cytoplasm" = "#00D55B", 
            "Nuclei" = "#0000AB"
            )
    )
)
row_measurement = rowAnnotation(
    FeatureType = features$Measurement, 
           annotation_legend_param = list(
        title_position = "topcenter", 
        title_gp = gpar(fontsize = 16, angle = 0, fontface = "bold", hjust = 0.5),
        labels_gp = gpar(fontsize = 16, 
        title = gpar(fontsize = 16))),
    annotation_name_side = "bottom",
    annotation_name_gp = gpar(fontsize = 16),
    col = list(
            FeatureType = c(
            "AreaShape" = brewer.pal(8, "Paired")[1],
            "Correlation" = brewer.pal(8, "Paired")[2],
            "Granularity" = brewer.pal(8, "Paired")[3],
            "Intensity" = brewer.pal(8, "Paired")[4],
            "Location" = brewer.pal(8, "Paired")[5],
            "Neighbors" =  brewer.pal(8, "Paired")[6],
            "RadialDistribution" = brewer.pal(8, "Paired")[7],
            "Texture" = brewer.pal(8, "Paired")[8]
        )
    ),
    show_legend = TRUE
)
row_channel = rowAnnotation(
    Channel = features$Channel,
        annotation_legend_param = list(
        title_position = "topcenter", 
        title_gp = gpar(fontsize = 16, angle = 0, fontface = "bold", hjust = 0.5),
        labels_gp = gpar(fontsize = 16, 
        # make annotation bar text bigger
        legend = gpar(fontsize = 16),
        annotation_name = gpar(fontsize = 16),
        legend_height = unit(20, "cm"),
        legend_width = unit(1, "cm"),
        # make legend taller
        legend_height = unit(10, "cm"),
        legend_width = unit(1, "cm"),
        legend_key = gpar(fontsize = 16)
        )
    ),


        
    annotation_name_side = "bottom",
    # make font size bigger
    annotation_name_gp = gpar(fontsize = 16),
    col = list(
    Channel = c(
            "DNA" = "#0000AB",
            "CL 488_1" = "#B000B0",
            "CL 488_2" = "#00D55B",
            "CL 561" = "#FFFF00",
            "None" = "#B09FB0")
    )
)
row_annotations = c(row_compartment, row_measurement, row_channel)

In [6]:
list_of_mats_for_heatmaps <- list()
list_of_heatmaps <- list()
heatmap_list <- NULL
ht_opt(RESET = TRUE)
df$Metadata_dose <- as.numeric(df$Metadata_dose)
for (dose in unique(df$Metadata_dose)) {
    # check if the last in the number of doses

    # get the first dose
    single_dose_df <- df %>%
        filter(Metadata_dose == dose) %>%
        group_by(Metadata_Time) %>%
        select(-Metadata_Well, -Metadata_dose) %>% 
        summarise(across(everything(), ~ mean(., na.rm = TRUE))) %>%
        ungroup()

    # sort the columns by Metadata_Time
    single_dose_df <- single_dose_df %>% 
        select(Metadata_Time, everything()) %>%
        arrange(Metadata_Time)
    
    mat <- t(as.matrix(single_dose_df))
    
    colnames(mat) <- single_dose_df$Metadata_Time
    mat <- mat[-1,]
    
    if (dose == max(unique(df$Metadata_dose))) {
    
        heatmap_plot <- Heatmap(
            mat, 
            col = col_fun,
            show_row_names = FALSE,
            show_column_names = FALSE,
            cluster_columns = FALSE,
            column_names_gp = gpar(fontsize = 16), # Column name label formatting
            row_names_gp = gpar(fontsize = 14), 

            show_heatmap_legend = TRUE,
            heatmap_legend_param = list(
                        title = "Feature\nValue",
                        title_position = "topcenter", 
                        # direction = "horizontal",
                        title_gp = gpar(fontsize = 16, angle = 0, fontface = "bold", hjust = 1.0),
                        labels_gp = gpar(fontsize = 16),
                        legend_height = unit(4, "cm"),
                        legend_width = unit(3, "cm"),
                        annotation_legend_side = "bottom"
                        ), 
            row_dend_width = unit(2, "cm"),
            column_title = paste0("Dose: ", dose," uM"),
            # add the row annotations
            right_annotation = row_annotations,
            top_annotation = column_anno
        )
    } else {
        heatmap_plot <- Heatmap(
            mat, 
            col = col_fun,
            show_row_names = FALSE,
            cluster_columns = FALSE,
            show_column_names = FALSE,

            column_names_gp = gpar(fontsize = 16), # Column name label formatting
            row_names_gp = gpar(fontsize = 14), 

            show_heatmap_legend = FALSE,
            heatmap_legend_param = list(
                        title = "Feature\nValue",
                        title_position = "topcenter", 
                        title_gp = gpar(fontsize = 16, angle = 0, fontface = "bold", hjust = 1.0),
                        labels_gp = gpar(fontsize = 16),
                        legend_height = unit(4, "cm"),
                        legend_width = unit(3, "cm"),
                        annotation_legend_side = "bottom"
                        ), 
            row_dend_width = unit(2, "cm"),
            column_title = paste0("Dose: ", dose," uM"),
            top_annotation = column_anno,
        )
    }
    # add the heatmap to the list
    heatmap_list <- heatmap_list + heatmap_plot
}

In [7]:
width <- 20
height <- 12
options(repr.plot.width = width, repr.plot.height = height)
ht_opt$message = FALSE
png(filename = paste0(figure_path, "filtered_features.png"), width = width, height = height, units = "in", res = 600)
draw(
    heatmap_list,
    merge_legends = TRUE,
    
    heatmap_legend_side = "right",
    annotation_legend_side = "bottom"
)
dev.off()

png 
  2